# 🧠 Treinamento YOLOv5 - Emoções e Times

Este notebook treina dois modelos YOLOv5:
- **Modelo 1**: Reconhecimento de emoções faciais
- **Modelo 2**: Reconhecimento de escudos de times de futebol

## 📋 Estrutura do Dataset
```
images/
├── faces/
│   ├── train/
│   │   ├── images/
│   │   └── labels/
│   └── test/
│       ├── images/
│       └── labels/
└── teams/
    ├── train/
    │   ├── images/
    │   └── labels/
    └── test/
        ├── images/
        └── labels/
```

## 🔧 1. Configuração Inicial e Dependências

In [ ]:
# Verificar GPU
!nvidia-smi

# Instalar dependências
!pip install ultralytics roboflow supervision

# Clonar YOLOv5 (caso necessário)
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5
!pip install -r requirements.txt

import torch
import os
import shutil
from pathlib import Path
import yaml
from IPython.display import Image, display
import matplotlib.pyplot as plt

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")
print(f"GPU count: {torch.cuda.device_count()}")
if torch.cuda.is_available():
    print(f"GPU name: {torch.cuda.get_device_name(0)}")

## 💾 2. Conectar ao Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Definir caminhos base
DRIVE_PATH = '/content/drive/MyDrive'
PROJECT_PATH = f'{DRIVE_PATH}/YOLOv5_Models'
DATASET_PATH = f'{PROJECT_PATH}/images'

# Criar estrutura de pastas se não existir
os.makedirs(PROJECT_PATH, exist_ok=True)
os.makedirs(f'{PROJECT_PATH}/models', exist_ok=True)

print(f"📁 Projeto salvo em: {PROJECT_PATH}")
print(f"🗂️ Datasets em: {DATASET_PATH}")
print(f"🤖 Modelos serão salvos em: {PROJECT_PATH}/models")

## 📊 3. Preparação dos Datasets

In [ ]:
# Verificar estrutura dos dados
def verify_dataset_structure(base_path, dataset_name):
    """
    Verifica se a estrutura do dataset está correta
    """
    dataset_path = f"{base_path}/{dataset_name}"
    
    print(f"🔍 Verificando dataset: {dataset_name}")
    
    for split in ['train', 'test']:
        images_path = f"{dataset_path}/{split}/images"
        labels_path = f"{dataset_path}/{split}/labels"
        
        if os.path.exists(images_path) and os.path.exists(labels_path):
            img_count = len([f for f in os.listdir(images_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
            lbl_count = len([f for f in os.listdir(labels_path) if f.endswith('.txt')])
            
            print(f"  📂 {split}: {img_count} imagens, {lbl_count} labels")
            
            if img_count != lbl_count:
                print(f"  ⚠️ Aviso: Número de imagens e labels não coincidem em {split}")
        else:
            print(f"  ❌ Pasta {split} não encontrada ou incompleta")
    
    return True

# Verificar ambos os datasets
verify_dataset_structure(DATASET_PATH, 'faces')
print("\n" + "="*50 + "\n")
verify_dataset_structure(DATASET_PATH, 'teams')

## 😀 4. Configuração para Emoções Faciais

In [ ]:
# Configuração do dataset de emoções
faces_config = {
    'train': f'{DATASET_PATH}/faces/train/images',
    'val': f'{DATASET_PATH}/faces/test/images',  # Usando test como validação
    'nc': 5,  # Número de classes
    'names': ['anger', 'fear', 'happy', 'neutral', 'sad']  # Ordem exata do data.yaml
}

# Salvar configuração
faces_yaml_path = '/content/faces_dataset.yaml'
with open(faces_yaml_path, 'w') as f:
    yaml.dump(faces_config, f, default_flow_style=False)

print("📝 Configuração do dataset de emoções:")
print(yaml.dump(faces_config, default_flow_style=False))

# Verificar algumas imagens de exemplo
sample_images = list(Path(faces_config['train']).glob('*.jpg'))[:3]
print(f"\n🖼️ Exemplos de imagens de emoções ({len(sample_images)} de {len(list(Path(faces_config['train']).glob('*.jpg')))} total):")
for img_path in sample_images:
    print(f"  📷 {img_path.name}")
    display(Image(str(img_path), width=150))

## 🚀 5. Treinamento - Modelo de Emoções

In [ ]:
# Parâmetros de treinamento para emoções
EMOTIONS_EPOCHS = 100
EMOTIONS_BATCH_SIZE = 16
EMOTIONS_IMG_SIZE = 640

print("🎭 Iniciando treinamento do modelo de emoções...")
print(f"📊 Épocas: {EMOTIONS_EPOCHS}")
print(f"📦 Batch size: {EMOTIONS_BATCH_SIZE}")
print(f"🖼️ Tamanho da imagem: {EMOTIONS_IMG_SIZE}")

# Comando de treinamento
!python train.py \
  --img {EMOTIONS_IMG_SIZE} \
  --batch {EMOTIONS_BATCH_SIZE} \
  --epochs {EMOTIONS_EPOCHS} \
  --data {faces_yaml_path} \
  --weights yolov5s.pt \
  --project {PROJECT_PATH}/runs \
  --name emotions_model \
  --cache \
  --device 0

print("\n✅ Treinamento de emoções concluído!")

## ⚽ 6. Configuração para Times de Futebol

In [ ]:
# Configuração do dataset de times
teams_config = {
    'train': f'{DATASET_PATH}/teams/train/images',
    'val': f'{DATASET_PATH}/teams/test/images',  # Usando test como validação
    'nc': 20,  # Número de classes (conforme data.yaml)
    'names': [
        'arsenal', 'aston-villa-new', 'bournemouth', 'brentford', 'brighton', 
        'burnley', 'chelsea', 'crystal-palace', 'everton', 'fulham', 
        'liverpool', 'luton', 'mancity', 'manutd', 'newcastle', 
        'nottingham', 'sheffield', 'tottenham', 'westham', 'wolves'
    ]  # Ordem exata do data.yaml
}

# Salvar configuração
teams_yaml_path = '/content/teams_dataset.yaml'
with open(teams_yaml_path, 'w') as f:
    yaml.dump(teams_config, f, default_flow_style=False)

print("📝 Configuração do dataset de times:")
print(yaml.dump(teams_config, default_flow_style=False))

# Verificar algumas imagens de exemplo
sample_images = list(Path(teams_config['train']).glob('*.jpg'))[:3]
print(f"\n🖼️ Exemplos de imagens de times ({len(sample_images)} de {len(list(Path(teams_config['train']).glob('*.jpg')))} total):")
for img_path in sample_images:
    print(f"  📷 {img_path.name}")
    display(Image(str(img_path), width=150))

## 🚀 7. Treinamento - Modelo de Times

In [ ]:
# Parâmetros de treinamento para times
TEAMS_EPOCHS = 150
TEAMS_BATCH_SIZE = 16
TEAMS_IMG_SIZE = 640

print("⚽ Iniciando treinamento do modelo de times...")
print(f"📊 Épocas: {TEAMS_EPOCHS}")
print(f"📦 Batch size: {TEAMS_BATCH_SIZE}")
print(f"🖼️ Tamanho da imagem: {TEAMS_IMG_SIZE}")

# Comando de treinamento
!python train.py \
  --img {TEAMS_IMG_SIZE} \
  --batch {TEAMS_BATCH_SIZE} \
  --epochs {TEAMS_EPOCHS} \
  --data {teams_yaml_path} \
  --weights yolov5s.pt \
  --project {PROJECT_PATH}/runs \
  --name teams_model \
  --cache \
  --device 0

print("\n✅ Treinamento de times concluído!")

## 📊 8. Avaliação dos Modelos

In [ ]:
# Encontrar os melhores modelos
emotions_best = f"{PROJECT_PATH}/runs/emotions_model/weights/best.pt"
teams_best = f"{PROJECT_PATH}/runs/teams_model/weights/best.pt"

print("📈 Resultados do treinamento:")
print("\n🎭 MODELO DE EMOÇÕES:")
if os.path.exists(emotions_best):
    print(f"✅ Modelo salvo: {emotions_best}")
    
    # Mostrar resultados
    results_path = f"{PROJECT_PATH}/runs/emotions_model/results.png"
    if os.path.exists(results_path):
        display(Image(results_path, width=800))
else:
    print("❌ Modelo de emoções não encontrado")

print("\n⚽ MODELO DE TIMES:")
if os.path.exists(teams_best):
    print(f"✅ Modelo salvo: {teams_best}")
    
    # Mostrar resultados
    results_path = f"{PROJECT_PATH}/runs/teams_model/results.png"
    if os.path.exists(results_path):
        display(Image(results_path, width=800))
else:
    print("❌ Modelo de times não encontrado")

## 🧪 9. Teste dos Modelos

In [ ]:
import torch
from PIL import Image as PILImage
import matplotlib.pyplot as plt

def test_model(model_path, test_image_path, class_names):
    """
    Testa um modelo YOLOv5 em uma imagem
    """
    # Carregar modelo
    model = torch.hub.load('ultralytics/yolov5', 'custom', path=model_path, force_reload=True)
    model.eval()
    
    # Fazer predição
    results = model(test_image_path)
    
    # Mostrar resultados
    results.show()
    
    # Detalhes da predição
    predictions = results.pandas().xyxy[0]
    if len(predictions) > 0:
        for idx, pred in predictions.iterrows():
            class_name = class_names[int(pred['class'])]
            confidence = pred['confidence']
            print(f"📊 Detectado: {class_name} (confiança: {confidence:.2f})")
    else:
        print("❌ Nenhuma detecção encontrada")

# Testar modelo de emoções
if os.path.exists(emotions_best):
    print("🎭 Testando modelo de emoções:")
    # Pegar uma imagem de teste
    test_faces = list(Path(f'{DATASET_PATH}/faces/test/images').glob('*.jpg'))
    if test_faces:
        test_model(emotions_best, str(test_faces[0]), faces_config['names'])

print("\n" + "="*50 + "\n")

# Testar modelo de times
if os.path.exists(teams_best):
    print("⚽ Testando modelo de times:")
    # Pegar uma imagem de teste
    test_teams = list(Path(f'{DATASET_PATH}/teams/test/images').glob('*.jpg'))
    if test_teams:
        test_model(teams_best, str(test_teams[0]), teams_config['names'])

## 💾 10. Exportar Modelos para Google Drive

In [ ]:
def export_model_to_drive(source_path, destination_name):
    """
    Copia modelo treinado para pasta específica no Drive
    """
    if os.path.exists(source_path):
        destination_path = f"{PROJECT_PATH}/models/{destination_name}"
        shutil.copy2(source_path, destination_path)
        
        # Verificar tamanho do arquivo
        size_mb = os.path.getsize(destination_path) / (1024 * 1024)
        
        print(f"✅ {destination_name} exportado com sucesso!")
        print(f"📁 Caminho: {destination_path}")
        print(f"📊 Tamanho: {size_mb:.1f} MB")
        return destination_path
    else:
        print(f"❌ Modelo não encontrado: {source_path}")
        return None

print("📦 Exportando modelos treinados para Google Drive...")
print("\n🎭 MODELO DE EMOÇÕES:")
emotions_exported = export_model_to_drive(emotions_best, "emotion_model.pt")

print("\n⚽ MODELO DE TIMES:")
teams_exported = export_model_to_drive(teams_best, "team_model.pt")

print("\n" + "="*50)
print("🎉 TREINAMENTO CONCLUÍDO COM SUCESSO!")
print("\n📋 RESUMO DOS MODELOS:")

if emotions_exported:
    print(f"🎭 Emoções: {emotions_exported}")
    print(f"   Classes: {', '.join(faces_config['names'])}")

if teams_exported:
    print(f"⚽ Times: {teams_exported}")
    print(f"   Classes: {', '.join(teams_config['names'])}")

print("\n💡 PRÓXIMOS PASSOS:")
print("1. Baixe os arquivos .pt do Google Drive")
print("2. Coloque-os nas pastas dos containers:")
print("   - emotion_model.pt → ai-face-service/models/")
print("   - team_model.pt → ai-team-service/models/")
print("3. Execute o sistema distribuído com docker-compose")

## ⬇️ 11. Download Direto dos Modelos

In [ ]:
from google.colab import files

print("⬇️ Fazendo download dos modelos treinados...")

# Download do modelo de emoções
if emotions_exported and os.path.exists(emotions_exported):
    print("\n🎭 Baixando modelo de emoções...")
    files.download(emotions_exported)

# Download do modelo de times
if teams_exported and os.path.exists(teams_exported):
    print("\n⚽ Baixando modelo de times...")
    files.download(teams_exported)

print("\n✅ Downloads iniciados! Verifique a pasta de downloads do seu navegador.")